In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import csv
import copy
import argparse
import itertools
from collections import Counter
from collections import deque

import cv2 as cv
import mediapipe as mp

from tensorflow import keras


c:\Anaconda\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [3]:
data = np.zeros((0,33))
cnt = 0
for j in range(4):
    for i in range(7):
        data_path = f'./model/point_history_classifier/{j}/point_history_{i}.csv'
        tmp = pd.read_csv(data_path)
        tmp = np.array(tmp)
        cnt += tmp.shape[0]
        data = np.vstack((data,tmp))
data = np.array(data)

In [4]:
label = data[::21,0].flatten()
data = data[:,1:].reshape(int(cnt/21),1,21,32)
label = keras.utils.to_categorical(label, 7)

In [5]:
# 21X32 -> 16X42 변환
hdata = np.zeros((int(cnt/21),1,16,42))
for i in range(int(cnt/21)):
    for j in range(32):
        k = 0
        if j%2:
            k=1
        hdata[i,0,j//2,k::2] = data[i,0,:,j]
data = hdata

In [6]:
label = label.reshape(int(cnt/21),1,7).astype(int)

In [7]:
for i in range(hdata.shape[0]):
    for j in range(15,0,-1):
        hdata[i,0,j,:] -= hdata[i,0,j-1,:]

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((hdata,label)).shuffle(buffer_size=int(cnt/21))

In [9]:
dataset = train_dataset

In [10]:
def is_test(x, y):
    return x % 4 == 0

def is_train(x, y):
    return not is_test(x, y)

recover = lambda x,y: y

test_dataset = dataset.enumerate() \
                    .filter(is_test) \
                    .map(recover)

train_dataset = dataset.enumerate() \
                    .filter(is_train) \
                    .map(recover)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [46]:
dnn= tf.keras.models.Sequential([
    tf.keras.layers.Dense(64,input_shape = (16,42), activation = 'ReLU'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation = 'ReLU'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation = 'ReLU'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(256, activation = 'ReLU'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation = 'ReLU'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),                         
    tf.keras.layers.Dense(7, activation='softmax'),   
])


dnn.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [47]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=10)
check_point = tf.keras.callbacks.ModelCheckpoint(filepath='./DNN/{epoch}_{accuracy:.4f}.h5', monitor='loss', save_best_only=True, verbose=1)
dnn.fit(train_dataset, epochs = 32, callbacks=[es, check_point])

Epoch 1/32
  17504/Unknown - 38s 2ms/step - loss: 0.5800 - accuracy: 0.7786
Epoch 1: loss improved from inf to 0.57972, saving model to ./DNN\1_0.7787.h5
17512/17512 [==============================] - 38s 2ms/step - loss: 0.5797 - accuracy: 0.7787
Epoch 2/32
17493/17512 [============================>.] - ETA: 0s - loss: 0.2837 - accuracy: 0.9032
Epoch 2: loss improved from 0.57972 to 0.28373, saving model to ./DNN\2_0.9033.h5
17512/17512 [==============================] - 38s 2ms/step - loss: 0.2837 - accuracy: 0.9033
Epoch 3/32
17501/17512 [============================>.] - ETA: 0s - loss: 0.2212 - accuracy: 0.9286
Epoch 3: loss improved from 0.28373 to 0.22134, saving model to ./DNN\3_0.9286.h5
17512/17512 [==============================] - 41s 2ms/step - loss: 0.2213 - accuracy: 0.9286
Epoch 4/32
17497/17512 [============================>.] - ETA: 0s - loss: 0.1967 - accuracy: 0.9390
Epoch 4: loss improved from 0.22134 to 0.19670, saving model to ./DNN\4_0.9390.h5
17512/17512 [=====

In [54]:
dnn = tf.keras.models.load_model('./dnn/7_0.9505.h5')

In [55]:
dnn.evaluate(test_dataset)

5838/5838 [==============================] - 15s 3ms/step - loss: 0.0390 - accuracy: 0.9875


[0.03904910385608673, 0.9874957203865051]

In [11]:
model= tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units = 50, return_sequences=True ,input_shape = (16,42)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 50),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7, activation='softmax'),   
    tf.keras.layers.Flatten()                         
])


model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [12]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=10)
check_point = tf.keras.callbacks.ModelCheckpoint(filepath='./motionmodel/{epoch}_{accuracy:.4f}.h5', monitor='loss', save_best_only=True, verbose=1)
model.fit(train_dataset, epochs = 32, callbacks=[es, check_point])

Epoch 1/32
  25834/Unknown - 216s 8ms/step - loss: 0.3267 - accuracy: 0.8862
Epoch 1: loss improved from inf to 0.32667, saving model to ./motionmodel\1_0.8862.h5
25838/25838 [==============================] - 218s 8ms/step - loss: 0.3267 - accuracy: 0.8862
Epoch 2/32
25836/25838 [============================>.] - ETA: 0s - loss: 0.0592 - accuracy: 0.9836
Epoch 2: loss improved from 0.32667 to 0.05922, saving model to ./motionmodel\2_0.9836.h5
25838/25838 [==============================] - 218s 8ms/step - loss: 0.0592 - accuracy: 0.9836
Epoch 3/32
25838/25838 [==============================] - ETA: 0s - loss: 0.0352 - accuracy: 0.9907
Epoch 3: loss improved from 0.05922 to 0.03523, saving model to ./motionmodel\3_0.9907.h5
25838/25838 [==============================] - 218s 8ms/step - loss: 0.0352 - accuracy: 0.9907
Epoch 4/32
25834/25838 [============================>.] - ETA: 0s - loss: 0.0279 - accuracy: 0.9928
Epoch 4: loss improved from 0.03523 to 0.02787, saving model to ./motion

In [11]:
model = tf.keras.models.load_model('./motionmodel/25_0.9985.h5')

In [12]:
model.evaluate(test_dataset)

8613/8613 [==============================] - 21s 2ms/step - loss: 0.0014 - accuracy: 0.9995


[0.001400164095684886, 0.9995355606079102]

In [29]:
model_l= tf.keras.models.Sequential([
    tf.keras.layers.LSTM(units = 16, return_sequences=True ,input_shape = (16,42)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units = 16),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7, activation='softmax'),   
    tf.keras.layers.Flatten()                         
])


model_l.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [30]:
es = tf.keras.callbacks.EarlyStopping(monitor='loss',patience=10)
check_point = tf.keras.callbacks.ModelCheckpoint(filepath='./Lstm/{epoch}_{accuracy:.4f}.h5', monitor='loss', save_best_only=True, verbose=1)
model_l.fit(train_dataset, epochs = 32, callbacks=[es, check_point])

Epoch 1/32
  17507/Unknown - 125s 7ms/step - loss: 0.5290 - accuracy: 0.8126
Epoch 1: loss improved from inf to 0.52900, saving model to ./Lstm\1_0.8126.h5
17512/17512 [==============================] - 125s 7ms/step - loss: 0.5290 - accuracy: 0.8126
Epoch 2/32
17510/17512 [============================>.] - ETA: 0s - loss: 0.1678 - accuracy: 0.9461
Epoch 2: loss improved from 0.52900 to 0.16779, saving model to ./Lstm\2_0.9461.h5
17512/17512 [==============================] - 106s 6ms/step - loss: 0.1678 - accuracy: 0.9461
Epoch 3/32
17511/17512 [============================>.] - ETA: 0s - loss: 0.1178 - accuracy: 0.9637
Epoch 3: loss improved from 0.16779 to 0.11777, saving model to ./Lstm\3_0.9637.h5
17512/17512 [==============================] - 121s 7ms/step - loss: 0.1178 - accuracy: 0.9637
Epoch 4/32
17506/17512 [============================>.] - ETA: 0s - loss: 0.0873 - accuracy: 0.9737
Epoch 4: loss improved from 0.11777 to 0.08732, saving model to ./Lstm\4_0.9737.h5
17512/1751

In [31]:
model_l = tf.keras.models.load_model('./Lstm/32_0.9960.h5')

In [32]:
model_l.evaluate(test_dataset)

5838/5838 [==============================] - 13s 2ms/step - loss: 0.0172 - accuracy: 0.9949


[0.017216941341757774, 0.9948612451553345]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 16, 50)            18600     
                                                                 
 dropout (Dropout)           (None, 16, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 7)                 357       
                                                                 
 flatten (Flatten)           (None, 7)                 0         
                                                                 
Total params: 39,157
Trainable params: 39,157
Non-traina